In [18]:
%load_ext dotenv
%dotenv

import os
import openai

MODEL = "gpt-4o-mini"

openai.api_key = os.getenv("OPENAI_API_KEY")


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
from pydantic import BaseModel


domain = "Everything related to drinks. How they are created, what's in them, variations, everything."

class DataProperty(BaseModel):


class Class(BaseModel):
    description: str
    name: str

class Classes(BaseModel):
    chain_of_thought: str
    classes: list[Class]

tlc = openai.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": f"You are an ontology engineer, tasked with creating an ontology for the domain of <domain>{domain}</domain>. First, please provide the names of all classes that should be included in the second-highest level of the ontology, being subclasses of 'Thing'.",
        }
    ],
    response_format=Classes
).choices[0].message.parsed.model_dump_json(indent=2)

print(tlc)


{
  "chain_of_thought": "In the domain of drinks, it's pertinent to categorize aspects relating to types of drinks, ingredients, preparation methods, and variations. This leads to the identification of relevant classes that encapsulate these features.",
  "classes": [
    {
      "description": "Different types of alcoholic beverages, including beer, wine, and spirits.",
      "name": "AlcoholicDrink"
    },
    {
      "description": "Non-alcoholic beverages such as juices, soda, and water.",
      "name": "NonAlcoholicDrink"
    },
    {
      "description": "Ingredients used in various drinks such as fruits, herbs, sweeteners, and flavorings.",
      "name": "Ingredient"
    },
    {
      "description": "Methods and techniques used to prepare drinks, such as brewing, mixing, and steeping.",
      "name": "PreparationMethod"
    },
    {
      "description": "Variations and styles of drinks, like cocktails, smoothies, and infusions.",
      "name": "DrinkVariation"
    },
    {
    